# GSM-EVP


In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from IPython.display import display, Math, Markdown
from bmcs_matmod.api import GSM1D_EVP
gsm = GSM1D_EVP()
display(Markdown(gsm.latex_potentials()))

## Linear strain-driven loading

In [ ]:
E_val = 30e+3  # Young's modulus in MPa
K_val = 10e+3  # Bulk modulus in MPa
f_c_val = 30 # Strength in MPa
mparams = dict(E=E_val, K=K_val, f_c=f_c_val, eta_vp=10e+3)

f_time = 2  # Final time in secondsfinal_time = 1
t_t = np.linspace(0, f_time, 100)
eps_n_t = np.linspace(0, 0.01, 100)

response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t_ec, eps_ta_ec, sig_ta_ec, Eps_t_ec, Sig_t_ec, iter_t_ec, lam_t_ec, (d_t_t_ec, d_eps_ta_ec) = response
eps_t_ec = eps_ta_ec[:, 0]
sig_t_ec = sig_ta_ec[:, 0, 0]
# use the stress history to reproduce the response using stress control
response = gsm.get_G_response(sig_t_ec, t_t, **mparams)
t_t_sc, eps_ta_sc, sig_ta_sc, Eps_t_sc, Sig_t_sc, iter_t_sc, lam_t_sc, (d_t_t_sc, d_eps_ta_sc) = response
iter_t = iter_t_sc[:, 0]
eps_t_sc = eps_ta_sc[:, 0, 0]
sig_t_sc = sig_ta_sc[:, 0]
eps_t_ec.shape, sig_t_ec.shape, eps_t_sc.shape, sig_t_sc.shape

In [ ]:
eps_vp_t_ec, z_t_ec = Eps_t_ec[:, 0, :].T
sig_vp_t_ec, Z_t_ec = Sig_t_ec[:, 0, :].T
eps_vp_t_sc, z_t_sc = Eps_t_sc[:, 0, :].T
sig_vp_t_sc, Z_t_sc = Sig_t_sc[:, 0, :].T
sig_vp_t_ec.shape, eps_vp_t_ec.shape, sig_vp_t_sc.shape, eps_vp_t_sc.shape

In [ ]:
fig, ((ax_sig, ax_omega), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))
fig.canvas.header_visible = False

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

# Stress plots: now using "vp" (viscoplastic) instead of "ve"
ax_sig.plot(eps_t_ec, sig_t_ec, color='cadetblue', lw=5, label=r'$\varepsilon$', alpha=0.5)
ax_sig.plot(eps_t_ec - eps_vp_t_ec, sig_t_ec, ls='dashed', color='cadetblue',
            label=r'$\varepsilon-\varepsilon^\mathrm{vp}$')

ax_sig.plot(eps_t_sc, sig_t_sc, color='red', label=r'$\varepsilon$')
ax_sig.plot(eps_t_sc - eps_vp_t_sc, sig_t_sc, ls='dashed', color='red',
            label=r'$\varepsilon-\varepsilon^\mathrm{vp}$')

ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

# Strain plots: replace eps_p_t with eps_vp_t and update labels accordingly
ax_eps.plot(t_t_ec, eps_t_ec, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t_ec, eps_t_ec - eps_vp_t_ec, label=r'$\varepsilon - \varepsilon_{\mathrm{vp}}$',
            color='cadetblue', ls='dashed', lw=1)
ax_eps.fill_between(t_t_ec, eps_t_ec, eps_t_ec - eps_vp_t_ec, color='red', alpha=0.3,
                    label=r'$\varepsilon_{\mathrm{vp}}$')
ax_eps.fill_between(t_t_ec, eps_t_ec - eps_vp_t_ec, 0, color='green', alpha=0.3,
                    label=r'$\varepsilon - \varepsilon_{\mathrm{vp}} - \varepsilon^\mathrm{vp}$')
ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

# Second row: Lagrange multiplier and plastic strain plots
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t_ec, lam_t_ec[:, 0, :], 'o-', label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t_ec, eps_t_ec, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t_ec, Z_t_ec, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t_ec, eps_t_ec, label=r'$\varepsilon$', color='blue')
ax_z_eps_p.plot(t_t_ec, eps_vp_t_ec, label=r'$\varepsilon_{\mathrm{vp}}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_{\mathrm{vp}}$', color='blue')
ax_z_eps_p.legend()

plt.tight_layout()